### Import libraries and dataset

In [ ]:
import numpy
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.np_utils import to_categorical

### Preprocess data, normalize images

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

print("shape of X_train: ", X_train.shape)
print("shape of X_test: ", X_test.shape)

### Create CNN model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

def init():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

modelInfo = init()
modelInfo.summary()

### Train model for 1 epoch and return with current loss

In [ ]:
def train(models):
  
    losses = []
     
    for i in range(len(models)):
        history = models[i].fit(x=X_train,y=y_train, epochs=1, validation_data=(X_test, y_test))
        losses.append(round(history.history['loss'][-1], 4))
        
    return models, losses


In [ ]:
no_of_generations = 7
no_of_individuals = 10
mutate_factor = 0.05
individuals = []

layers = [1, 3]

In [ ]:
def mutate(new_individual):
    for layer in layers:
        for bias in range(len(new_individual.layers[layer].get_weights()[1])):
            n = random.random()
            if n < mutate_factor:
                new_individual.layers[layer].get_weights(
                )[1][bias] *= random.uniform(-0.5, 0.5)

    for layer in layers:
        for weight in new_individual.layers[layer].get_weights()[0]:
            n = random.random()
            if n < mutate_factor:
                for j in range(len(weight)):
                    if random.random() < mutate_factor:
                        new_individual.layers[layer].get_weights(
                        )[0][j] *= random.uniform(-0.5, 0.5)

    return new_individual

In [ ]:
def crossover(individuals_param):
    new_individuals = [individuals_param[0], individuals_param[1]]

    for j in range(2, no_of_individuals):
        if j < (no_of_individuals - 2):
            if j == 2:
                parentA = random.choice(individuals_param[:3])
                parentB = random.choice(individuals_param[:3])
            else:
                parentA = random.choice(individuals_param[:])
                parentB = random.choice(individuals_param[:])

            for j in layers:
                temp = parentA.layers[j].get_weights()[1]
                parentA.layers[j].get_weights()[1] = parentB.layers[j].get_weights()[1]
                parentB.layers[j].get_weights()[1] = temp

            new_individual = random.choice([parentA, parentB])

        else:
            new_individual = random.choice(individuals_param[:])

        new_individuals.append(mutate(new_individual))
        # new_individuals.append(new_individual)

    return new_individuals

In [ ]:
def evolve(individuals_param, losses_param):
    sorted_y_idx_list = sorted(range(len(losses_param)), key=lambda x: losses_param[x])
    individuals_param = [individuals_param[x] for x in sorted_y_idx_list]

    # winners = individuals[:6]

    new_individuals = crossover(individuals_param)

    return new_individuals


In [ ]:
for i in range(no_of_individuals):
        individuals.append(init())

for generation in range(no_of_generations):
    individuals, losses = train(individuals)
    print(losses)

    individuals = evolve(individuals, losses)

In [ ]:
model = individuals[0]
model.evaluate(X_test, y_test)

In [ ]:
p_test = model.predict(X_test).argmax(axis=1)
i = numpy.random.choice(X_test.shape[0], replace=False)
plt.imshow(X_test[i], cmap= 'gray')
plt.title("True label: %s Predicted: %s" % (y_test[i], p_test[i]))